In [38]:
import pandas
import geopandas
from shapely.geometry import Point

Eu utilizei somente os datasets que possuem dados de latitude e longitude inicialmente mas a ideia
é adicionar informações de bairro.

# Data Sources

As fontes de dados são do site dados.recife.pe.gov.br. Todas são relacionadas à CTTU

In [36]:
acidentes_2014 = pandas.read_csv("data/acidentes-2014.csv", sep=";")
acidentes_2015 = pandas.read_csv("data/acidentes-2015.csv", sep=";")
acidentes_2016 = pandas.read_csv("data/acidentes-2016.csv", sep=";")

In [5]:
equipamentos_monitoramento = pandas.read_csv("data/equipamentos-de-monitoramento-e-ficalizacao.csv", sep=";")

In [71]:
semaforos = pandas.read_csv("data/semaforos.csv", sep=";")

In [7]:
fiscalizacao = pandas.read_csv("data/fiscalizacao-eletronica.csv", sep=";")

In [30]:
monitoramento = pandas.read_csv("data/monitoramentocttu.csv", sep=";")


## Discussão
Um desafio significante do projeto foi juntar os dataframes a partir dos lat e longs, como juntar exemplos, 
como calcular a distância ótima para que acidentes não fossem inseridos próximos de dois semafóros, por exempo?



# Exploração - fiscalização

In [26]:
fiscalizacao.head(10)

,NOME,EQUIPAMENTO,CONTRATO,ENDERECO,FLUXO_VEICULO,VELOCIDADE,LONGITUDE,LATITUDE
0,1,DEV,015/2011,AVENIDA ENGº ABDIAS CARVALHO - PROX N 101 - SE...,29000.0,60.0,-34.935969,-8.063842
1,2,DEV,015/2011,AVENIDA ENGº ABDIAS CARVALHO - PROX N 101 - SE...,34000.0,60.0,-34.935493,-8.063640
2,3,DEV,015/2011,AV MAL MASCARENHAS DE MORAES - PROX AEROPORTO ...,64000.0,60.0,-34.915644,-8.130088
3,4,DEV,015/2011,AV MAL MASCARENHAS DE MORAES - PROX AEROPORTO ...,64000.0,60.0,-34.915615,-8.129942
4,5,DEV,015/2011,AV MAL MASCARENHAS DE MORAES - PROX AEROPORTO ...,64000.0,60.0,-34.915856,-8.130059
5,6,DEV,015/2011,AV MAL MASCARENHAS DE MORAES - PROX AEROPORTO ...,64000.0,60.0,-34.915409,-8.129972
6,7,DEV,015/2011,AV BOA VIAGEM - PROX N 102,43000.0,60.0,-34.882100,-8.091903
7,8,DEV,015/2011,TERCEIRO JARDIM/BOA VIAGEM,43000.0,60.0,-34.888333,-8.107912
8,9,DEV,015/2011,LARGO DO CABANGA - SENTIDO BOA VIAGEM,43000.0,60.0,-34.892030,-8.079941
9,10,DEV,015/2011,LARGO DO CABANGA - SENTIDO DERBY,57000.0,60.0,-34.890900,-8.079600


In [27]:
fiscalizacao.dtypes

NOME               int64
EQUIPAMENTO       object
CONTRATO          object
ENDERECO          object
FLUXO_VEICULO    float64
VELOCIDADE       float64
LONGITUDE        float64
LATITUDE         float64
dtype: object

In [23]:
fiscalizacao.shape

(35, 8)

In [21]:
fiscalizacao.describe()

,NOME,FLUXO_VEICULO,VELOCIDADE,LONGITUDE,LATITUDE
count,35.000000,28.000000,26.000000,35.000000,35.000000
mean,18.000000,41396.000000,53.846154,-34.905407,-8.079984
std,10.246951,16311.373246,9.413574,0.024032,0.033942
min,1.000000,14000.000000,40.000000,-34.958040,-8.153316
25%,9.500000,30500.000000,40.000000,-34.927739,-8.097473
50%,18.000000,40000.000000,60.000000,-34.908242,-8.074541
75%,26.500000,57000.000000,60.000000,-34.884947,-8.053121
max,35.000000,64000.000000,60.000000,-34.871560,-8.038163


# Exploração - Semaforos

In [25]:
semaforos.describe()

,semaforo,Latitude,Longitude
count,655.000000,655.000000,654.000000
mean,328.766412,-8.110816,-34.904928
std,190.417469,1.049914,0.019833
min,1.000000,-34.925727,-34.968542
25%,164.500000,-8.096992,-34.915793
50%,328.000000,-8.062029,-34.902649
75%,491.500000,-8.043514,-34.889812
max,663.000000,-8.004139,-34.870413


In [28]:
semaforos.shape

(655, 9)

In [29]:
semaforos.head(20)

,semaforo,localizacao1,localizacao2,funcionamento,utilizacao,sinalsonoro,sinalizadorciclista,Latitude,Longitude
0,1,"Boa Viagem, Av.",Em frente ao parque Dona Lindú,E/Git,Ocasional/Ped.,N,N,-8.142323,-34.903798
1,2,"Conselheiro Aguiar, Av.","Thomé Gibson, Rua",E/Git,Veicular,N,N,-8.099068,-34.885883
2,3,"Boa Viagem, Av.","França Pereira, Rua",E/Git,Ocasional/Ped.,N,S,-8.106806,-34.887726
3,4,"Boa Viagem, Av.","Barão de Souza Leão, Rua",E/Git,Ocasional/Ped.,N,S,-8.132230,-34.899990
4,5,"Conselheiro Aguiar, Av.","Artur Muniz, Rua",E/Git,Veicular,N,N,-8.104310,-34.888228
5,6,"Conselheiro Aguiar, Av.","Antônio Falcão, Rua",E/Git,Veicular,N,N,-8.116171,-34.894686
6,7,"Mascarenhas de Morais, Av.","Jamaica, Rua",E/Git,Veicular,N,N,-8.107621,-34.911660
7,8,"Caxangá, Av.","Bom Pastor, Rua do",E/Git,Veicular,N,N,-8.040820,-34.938494
8,9,"Cruz Cabugá, Av.",Est BRT Casa da Misericórdia,E/Git,Pedestre,N,N,-8.044224,-34.874595
9,10,II Perimetral,"Parnamirim, Av.",E/Git,Veicular,N,N,-8.034731,-34.908303


# Data Cleaning

## Monitoramento

In [39]:
monitoramento['location'] = monitoramento.apply(lambda example: Point(example.latitude, example.longitude), axis=1)

In [53]:
monitoramento = geopandas.GeoDataFrame(monitoramento)
monitoramento.head(5)

,nome,endereco,longitude,latitude,location
0,Câmera 150,Largo do Cabanga - Ponte do Pina,-34.891049,-8.080844,POINT (-8.080844000000001 -34.891049)
1,Câmera 151,Av. Herculano Bandeira - Semáforo 137,-34.885291,-8.088795,POINT (-8.088794999999999 -34.885291)
2,Câmera 152,Av. Agamenon Magalhães - sentido Olinda,-34.884789,-8.042955,POINT (-8.042955000000001 -34.884789)
3,Câmera 153,Av. Boa Viagem - 3º Jardim,-34.888551,-8.108781,POINT (-8.108781 -34.888551)
4,Câmera 154,Av Engenhriro Abdias de Carvalho - próx à lomb...,-34.934995,-8.063758,POINT (-8.063758 -34.934995)


Não achei necessário a transformação de nenhum tipo do dataset de monitoramento, apenas a transformação
do tipo para **GeoDataFrame** e a criação da coluna location

## Fiscalizacao

In [48]:
fiscalizacao["location"] = fiscalizacao.apply(lambda example: Point(example.LATITUDE, example.LONGITUDE), axis=1)

In [49]:
fiscalizacao = geopandas.GeoDataFrame(fiscalizacao)

In [51]:
fiscalizacao.dtypes

NOME               int64
EQUIPAMENTO       object
CONTRATO          object
ENDERECO          object
FLUXO_VEICULO    float64
VELOCIDADE       float64
LONGITUDE        float64
LATITUDE         float64
location          object
dtype: object

## Semáforos

In [60]:
semaforos.head(5)

,semaforo,localizacao1,localizacao2,funcionamento,utilizacao,sinalsonoro,sinalizadorciclista,Latitude,Longitude
0,1,"Boa Viagem, Av.",Em frente ao parque Dona Lindú,E/Git,Ocasional/Ped.,N,N,-8.142323,-34.903798
1,2,"Conselheiro Aguiar, Av.","Thomé Gibson, Rua",E/Git,Veicular,N,N,-8.099068,-34.885883
2,3,"Boa Viagem, Av.","França Pereira, Rua",E/Git,Ocasional/Ped.,N,S,-8.106806,-34.887726
3,4,"Boa Viagem, Av.","Barão de Souza Leão, Rua",E/Git,Ocasional/Ped.,N,S,-8.132230,-34.899990
4,5,"Conselheiro Aguiar, Av.","Artur Muniz, Rua",E/Git,Veicular,N,N,-8.104310,-34.888228


In [72]:
semaforos["location"] = semaforos.apply(lambda example: Point(example.Latitude, example.Longitude), axis=1)

In [73]:
semaforos = geopandas.GeoDataFrame(semaforos)

In [74]:
semaforos.dtypes

semaforo                 int64
localizacao1            object
localizacao2            object
funcionamento           object
utilizacao              object
sinalsonoro             object
sinalizadorciclista     object
Latitude               float64
Longitude              float64
location                object
dtype: object

In [75]:
categorical_columns = ["funcionamento", "sinalsonoro", "sinalizadorciclista"]
for column in categorical_columns:
    semaforos[column] = semaforos[column].astype('category')

In [76]:
semaforos.dtypes

semaforo                  int64
localizacao1             object
localizacao2             object
funcionamento          category
utilizacao               object
sinalsonoro            category
sinalizadorciclista    category
Latitude                float64
Longitude               float64
location                 object
dtype: object

In [64]:
semaforos.head(5)

,semaforo,localizacao1,localizacao2,funcionamento,utilizacao,sinalsonoro,sinalizadorciclista,Latitude,Longitude,location
0,1,"Boa Viagem, Av.",Em frente ao parque Dona Lindú,E/Git,Ocasional/Ped.,N,N,-8.142323,-34.903798,POINT (-8.142322999999999 -34.90379799999999)
1,2,"Conselheiro Aguiar, Av.","Thomé Gibson, Rua",E/Git,Veicular,N,N,-8.099068,-34.885883,POINT (-8.099067999999999 -34.885883)
2,3,"Boa Viagem, Av.","França Pereira, Rua",E/Git,Ocasional/Ped.,N,S,-8.106806,-34.887726,POINT (-8.106806000000001 -34.887726)
3,4,"Boa Viagem, Av.","Barão de Souza Leão, Rua",E/Git,Ocasional/Ped.,N,S,-8.132230,-34.899990,POINT (-8.13223 -34.89999)
4,5,"Conselheiro Aguiar, Av.","Artur Muniz, Rua",E/Git,Veicular,N,N,-8.104310,-34.888228,POINT (-8.10431 -34.88822800000001)


In [25]:
semaforos["funcionamento"].value_counts()

E/Git    498
E        138
E/GIt      3
E/GIT      1
Name: funcionamento, dtype: int64

In [27]:
semaforos["funcionamento"] = semaforos["funcionamento"].replace({'E/GIt': "E/Git", "E/GIT": "E/Git"})

In [28]:
semaforos["funcionamento"].value_counts()

E/Git    502
E        138
Name: funcionamento, dtype: int64

In [31]:
semaforos["sinalsonoro"].value_counts()

N    592
S     48
Name: sinalsonoro, dtype: int64

In [32]:
semaforos["sinalizadorciclista"].value_counts()

N            597
S             43
-8.107744      1
Name: sinalizadorciclista, dtype: int64

In [33]:
semaforos["localizacao2"].value_counts()

Gervásio Pires, Rua                                       5
Bruno Veloso, Rua                                         5
Félix de Brito, Rua                                       4
Remédios, Estrada dos                                     4
Barão de Souza Leão, Rua                                  4
Futuro, Rua do                                            4
Rua Nogueira de Souza                                     3
Thomé Gibson, Rua                                         3
Antônio Falcão, Rua                                       3
Fernando Simões Barbosa, Av.                              3
Peixoto, Rua do                                           3
Sá e Souza, Rua                                           3
Baltazar Passos, Rua                                      3
Pereira da Costa, Rua                                     3
Ernesto de Paula Santos, Rua                              3
48, Rua                                                   3
José Maria, Rua Dr.                     

Mesmo que os valores Veicular e Pedestre não estejam presentes no dicionário de dados, não 
há indicação para o descarte, então irei mantê-los

In [136]:
semaforos["utilizacao"].value_counts()

Veicular             316
Ocasional/Ped.       206
Veic.C/Ped.Ocas.      39
Veic.C/Ped.Paral.     36
Veic.C/Ped.           23
Pedestre              13
Veic.c/Ped.Ocas.       6
Veic.C/Laço Det.       1
Name: utilizacao, dtype: int64

## Acidentes

In [87]:
acidentes_2014.columns

Index(['data', 'tipo', 'detalhes', 'longitude', 'latitude'], dtype='object')

Percebi que a coluna tipo do dataset de 2014 é o mesmo tipo_de_ocorrencia de 2015 e 2016, então  a 
renomiei para "tipo de ocorrencia"

In [95]:
acidentes_2014["tipo"].value_counts()

Colisoes                659
Moto e Ciclomotor       300
Motocicletas            179
Atropelamentos          148
Motos e Ciclomotores    148
Automóveis e outros      85
Ciclistas e pedestre     47
Ciclistas                37
Pedestres e ciclista     35
Ciclistas e Pedestre     31
Pedestres                30
Ciclomotores             23
Automóveis                9
Outros                    3
Name: tipo, dtype: int64

In [85]:
acidentes_2015.columns

Index(['data_abertura', 'hora_abertura', 'bairro', 'endereco', 'complemento',
       'tipo_ocorrencia', 'quantidade_vitimas', 'descricao', 'tipo',
       'latitude', 'longitude'],
      dtype='object')

In [97]:
acidentes_2015["tipo"].value_counts()

Motocicleta     806
Pedestre        159
Motocicletas    150
Automóvel       121
Ciclomotor       59
Ciclista         55
Outros           19
Name: tipo, dtype: int64

In [37]:
map_2015_2016 = {"data_abertura": "data de abertura", "hora_abertura": "hora de abertura",
                "quantidade_vitimas": "quantidade de vitimas", "tipo_ocorrencia": "tipo de ocorrencia"}

In [38]:
map_2014_2016 = {"tipo": "tipo de ocorrencia"}

In [39]:
acidentes_2014 = acidentes_2014.rename(map_2014_2016, axis='columns')
acidentes_2014.columns

Index(['data', 'tipo de ocorrencia', 'detalhes', 'longitude', 'latitude'], dtype='object')

In [40]:
acidentes_2015 = acidentes_2015.rename(map_2015_2016, axis='columns')
acidentes_2015.columns

Index(['data de abertura', 'hora de abertura', 'bairro', 'endereco',
       'complemento', 'tipo de ocorrencia', 'quantidade de vitimas',
       'descricao', 'tipo', 'latitude', 'longitude'],
      dtype='object')

In [41]:
acidentes_2016.columns

Index(['longitude', 'latitude', 'data de abertura', 'hora de abertura',
       'bairro', 'endereco', 'complemento', 'tipo de ocorrencia',
       'quantidade de vitimas', 'descricao', 'tipo'],
      dtype='object')

In [89]:
acidentes_2016["tipo"].head(5)

0    Automóveis
1    Automóveis
2    Automóveis
3    Automóveis
4    Automóveis
Name: tipo, dtype: object

In [96]:
acidentes_2016["tipo de ocorrencia"].value_counts()

COLISÃO                   970
ATROPELAMENTO             147
COLISÃO COM CICLISTA       42
CHOQUE                     34
COLISÃOa                    8
CAPOTAMENTO                 5
ENTRADA E SAÍDA             4
TOMBAMENTO                  4
ENGAVETAMENTO               4
COLISÃO\t2016 13 050\t      2
ACID. DE PERCURSO           2
QUEDA DE ÁRVORE             1
ATROPELAMENTOa              1
ATROPELAMENTO ANIMAL        1
FISCALIZAÇÃO                1
Name: tipo de ocorrencia, dtype: int64

In [42]:
acidentes_2014_2016 = pandas.concat([acidentes_2014, acidentes_2015, acidentes_2016])

In [43]:
acidentes_2014_2016.shape

(4334, 13)

No dicionário de dados não há nada que mencione se há uma diferença entre "Moto e Ciclomotor" e "Motos e Ciclomotores", 
irei assumir que não há, dado que no máximo o que haverá será uma diferença na quantidade, se houver, 
acho que a perda de informação será mínima. 

Colocarei Motocicletas em separado pois acho que são uma categoria diferente de ciclomotor (como mostrado, há uma categoria "Ciclomotores"

In [65]:
acidentes_2014_2016["tipo de ocorrencia"].value_counts()

Colisão                 2763
Atropelamento            449
Moto e Ciclomotor        448
Motocicletas             179
Ciclistas e Pedestre     113
Automóveis e outros       85
COLISÃO COM CICLISTA      76
CHOQUE                    62
Ciclistas                 37
Pedestres                 30
Ciclomotores              23
CAPOTAMENTO               11
Automóveis                 9
ACID. DE PERCURSO          8
TOMBAMENTO                 8
ATROPELAMENTO ANIMAL       6
ENGAVETAMENTO              5
ENTRADA E SAÍDA            4
COLISÃO                    4
Outros                     3
QUEDA DE ÁRVORE            1
FISCALIZAÇÃO               1
ATROPELAMENTO              1
Name: tipo de ocorrencia, dtype: int64

In [49]:
default_colisao_cicl_pedestrians_value = "Ciclistas e Pedestre"
acidentes_2014_2016["tipo de ocorrencia"] = acidentes_2014_2016["tipo de ocorrencia"].replace({"Ciclistas e pedestre": default_colisao_cicl_pedestrians_value, 
                                                  "Pedestres e ciclista": default_colisao_cicl_pedestrians_value})

In [57]:
default_colisao = "Colisão"
acidentes_2014_2016["tipo de ocorrencia"] = acidentes_2014_2016["tipo de ocorrencia"].replace({"COLISÃO": default_colisao,
                                                                                              '"COLISÃO': default_colisao,
                                                                                              'Colisoes': default_colisao,
                                                                                              'COLISÃOa': default_colisao,
                                                                                              "COLISÃO\t2016 13 050\t": default_colisao,
                                                                                              "COLISÃO": default_colisao})

In [59]:
default_atropelamento = "Atropelamento"
acidentes_2014_2016["tipo de ocorrencia"] = acidentes_2014_2016["tipo de ocorrencia"].replace({
    "ATROPELAMENTOa": default_atropelamento,
    "ATROPELAMENTO": default_atropelamento,
    "Atropelamentos": default_atropelamento,
    
})

In [61]:
default_acid_percurso = "ACID. DE PERCURSO"
acidentes_2014_2016["tipo de ocorrencia"] = acidentes_2014_2016["tipo de ocorrencia"].replace({
    "ACID DE PERCURSO": default_acid_percurso
})

In [64]:
default_moto_ciclomotor = "Moto e Ciclomotor"
acidentes_2014_2016["tipo de ocorrencia"] = acidentes_2014_2016["tipo de ocorrencia"].replace({
    "Motos e Ciclomotores": default_moto_ciclomotor
})